In [ ]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import lightgbm as lgb
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error

trainData = pd.read_csv('train.csv')
testData = pd.read_csv('test.csv')

features_to_use = trainData.columns[2:18]
features_to_use

In [ ]:

# import pandas as pd
# import numpy as np
# from sklearn.model_selection import train_test_split
# from sklearn.preprocessing import RobustScaler
# from sklearn.metrics import mean_absolute_error
# import xgboost as xgb
# from lightgbm import LGBMRegressor
# from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
# import warnings
# warnings.filterwarnings('ignore')

# def load_and_preprocess():
#     train = pd.read_csv('train.csv')
#     test = pd.read_csv('test.csv')
#     test_ids = test['id'].copy()
#     all_data = pd.concat([train.drop('yield', axis=1), test], axis=0, ignore_index=True)
#     return train, test, all_data, test_ids

# def create_features(df):
#     df = df.copy()
    
#     # Core bee features - from best performing version
#     df['total_bees'] = df['honeybee'] + df['bumbles'] + df['andrena'] + df['osmia']
#     df['wild_bees'] = df['bumbles'] + df['andrena'] + df['osmia']
#     df['bee_ratio'] = df['honeybee'] / (df['total_bees'] + 1)
    
#     # Temperature features
#     df['upper_temp_range'] = df['MaxOfUpperTRange'] - df['MinOfUpperTRange']
#     df['lower_temp_range'] = df['MaxOfLowerTRange'] - df['MinOfLowerTRange']
#     df['avg_temp'] = (df['AverageOfUpperTRange'] + df['AverageOfLowerTRange']) / 2
    
#     # Fruit and pollination features
#     df['fruit_efficiency'] = df['fruitset'] * df['fruitmass']
#     df['seed_density'] = df['seeds'] / (df['fruitmass'] + 1)
#     df['clone_fruit'] = df['clonesize'] * df['fruitset']
    
#     # Weather interactions
#     df['rain_temp_effect'] = df['RainingDays'] * df['avg_temp']
#     df['bee_weather'] = df['total_bees'] * df['avg_temp'] * (1 / (df['RainingDays'] + 1))
    
#     # Log transforms for skewed features
#     for col in ['clonesize', 'fruitmass', 'seeds']:
#         df[f'log_{col}'] = np.log1p(df[col])
    
#     # Squared terms for key variables
#     for col in ['fruitset', 'seeds', 'fruit_efficiency']:
#         df[f'{col}_squared'] = df[col] ** 2
    
#     return df

# def train_models(X_train, y_train, X_val, y_val):
#     models = {
#         'xgb': xgb.XGBRegressor(
#             n_estimators=3000,
#             learning_rate=0.003,
#             max_depth=6,
#             subsample=0.85,
#             colsample_bytree=0.85,
#             min_child_weight=5,
#             gamma=0.1,
#             reg_alpha=0.1,
#             reg_lambda=1,
#             objective='reg:absoluteerror',
#             tree_method='hist',
#             random_state=42
#         ),
#         'lgbm': LGBMRegressor(
#             n_estimators=3000,
#             learning_rate=0.003,
#             num_leaves=64,
#             subsample=0.85,
#             colsample_bytree=0.85,
#             min_child_samples=20,
#             objective='mae',
#             random_state=42,
#             force_col_wise=True
#         ),
#         'rf': RandomForestRegressor(
#             n_estimators=500,
#             max_depth=15,
#             min_samples_split=5,
#             min_samples_leaf=4,
#             max_features='sqrt',
#             criterion='absolute_error',
#             random_state=42,
#             n_jobs=-1
#         ),
#         'gb': GradientBoostingRegressor(
#             n_estimators=500,
#             learning_rate=0.03,
#             max_depth=6,
#             min_samples_split=5,
#             min_samples_leaf=4,
#             subsample=0.8,
#             loss='absolute_error',
#             random_state=42
#         )
#     }
    
#     val_predictions = {}
#     trained_models = {}
    
#     for name, model in models.items():
#         model.fit(X_train, y_train)
#         val_pred = model.predict(X_val)
#         val_predictions[name] = val_pred
#         trained_models[name] = model
        
#         val_score = mean_absolute_error(y_val, val_pred)
#         print(f"{name} Validation MAE: {val_score:.4f}")
    
#     return trained_models, val_predictions

# def blend_predictions(val_predictions, y_val):
#     base_weights = {
#         'xgb': 0.35,
#         'lgbm': 0.30,
#         'rf': 0.15,
#         'gb': 0.20
#     }
    
#     best_mae = float('inf')
#     best_weights = base_weights.copy()
    
#     # Fine-tune around the base weights that worked well before
#     for xgb_w in np.arange(0.33, 0.37, 0.002):  # Centered around 0.35
#         for lgbm_w in np.arange(0.28, 0.32, 0.002):  # Centered around 0.30
#             for rf_w in np.arange(0.13, 0.17, 0.002):  # Centered around 0.15
#                 gb_w = 1 - xgb_w - lgbm_w - rf_w
#                 if 0.18 <= gb_w <= 0.22:  # Centered around 0.20
#                     weights = {
#                         'xgb': xgb_w,
#                         'lgbm': lgbm_w,
#                         'rf': rf_w,
#                         'gb': gb_w
#                     }
                    
#                     pred = np.zeros(len(y_val))
#                     for model, weight in weights.items():
#                         pred += val_predictions[model] * weight
                    
#                     mae = mean_absolute_error(y_val, pred)
#                     if mae < best_mae:
#                         best_mae = mae
#                         best_weights = weights.copy()
    
#     print(f"\nBest weights found: {best_weights}")
#     print(f"Best Validation MAE: {best_mae:.4f}")
#     return best_weights

# def main():
#     # Load data
#     train, test, all_data, test_ids = load_and_preprocess()
    
#     # Create features
#     all_data_featured = create_features(all_data)
    
#     # Drop unnecessary columns
#     drop_cols = ['id', 'Row#']
#     all_data_featured = all_data_featured.drop(drop_cols, axis=1)
    
#     # Split data
#     X = all_data_featured.iloc[:len(train)]
#     X_test = all_data_featured.iloc[len(train):]
#     y = train['yield']
    
#     X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)
    
#     # Scale features
#     scaler = RobustScaler()
#     X_train_scaled = scaler.fit_transform(X_train)
#     X_val_scaled = scaler.transform(X_val)
#     X_test_scaled = scaler.transform(X_test)
    
#     X_train_scaled = pd.DataFrame(X_train_scaled, columns=X_train.columns)
#     X_val_scaled = pd.DataFrame(X_val_scaled, columns=X_val.columns)
#     X_test_scaled = pd.DataFrame(X_test_scaled, columns=X_test.columns)
    
#     # Train models
#     trained_models, val_predictions = train_models(X_train_scaled, y_train, X_val_scaled, y_val)
    
#     # Find optimal weights
#     best_weights = blend_predictions(val_predictions, y_val)
    
#     # Generate predictions
#     test_predictions = {}
#     for name, model in trained_models.items():
#         test_predictions[name] = model.predict(X_test_scaled)
    
#     # Create final predictions
#     final_predictions = np.zeros(len(X_test))
#     for model, weight in best_weights.items():
#         final_predictions += test_predictions[model] * weight
    
#     # Create submission file
#     submission = pd.DataFrame({
#         'id': test_ids,
#         'yield': final_predictions
#     })
#     submission['id'] = submission['id'].astype(int)
#     submission = submission.sort_values('id').reset_index(drop=True)
    
#     submission.to_csv('submission.csv', index=False)
#     print("\nFirst few rows of submission file:")
#     print(submission.head())
#     print("\nSubmission file shape:", submission.shape)
#     print("Submission file created successfully!")

# if __name__ == "__main__":
#     main()

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import RobustScaler
from sklearn.metrics import mean_absolute_error
import xgboost as xgb
from lightgbm import LGBMRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
import warnings
warnings.filterwarnings('ignore')

def select_features(X, y, importance_threshold=0.001):
    """
    Select features using Random Forest importance scores
    
    Parameters:
    -----------
    X : DataFrame
        Features DataFrame
    y : Series
        Target variable
    importance_threshold : float
        Minimum importance score for feature selection
    """
    print("\nPerforming Random Forest feature selection...")
    
    # Initialize Random Forest for feature selection
    rf_selector = RandomForestRegressor(
        n_estimators=200,
        max_depth=10,
        random_state=42,
        n_jobs=-1
    )
    rf_selector.fit(X, y)
    
    # Get feature importances
    importances = pd.Series(
        rf_selector.feature_importances_,
        index=X.columns
    ).sort_values(ascending=False)
    
    # Select features based on importance threshold
    selected_features = importances[importances >= importance_threshold].index.tolist()
    
    # Print feature importance summary
    print("\nFeature Importances:")
    for feat, imp in importances.items():
        print(f"{feat}: {imp:.4f}")
    
    print(f"\nSelected {len(selected_features)} features out of {len(X.columns)}")
    print("Selected features:", selected_features)
    
    # Save feature importances to CSV
    importances.to_frame('importance').to_csv('feature_importances.csv')
    print("\nFeature importances saved to 'feature_importances.csv'")
    
    return selected_features

def load_and_preprocess():
    train = pd.read_csv('train.csv')
    test = pd.read_csv('test.csv')
    test_ids = test['id'].copy()
    all_data = pd.concat([train.drop('yield', axis=1), test], axis=0, ignore_index=True)
    return train, test, all_data, test_ids

def create_features(df):
    df = df.copy()
    
    # Core bee features
    df['total_bees'] = df['honeybee'] + df['bumbles'] + df['andrena'] + df['osmia']
    df['wild_bees'] = df['bumbles'] + df['andrena'] + df['osmia']
    df['bee_ratio'] = df['honeybee'] / (df['total_bees'] + 1)
    
    # Temperature features
    df['upper_temp_range'] = df['MaxOfUpperTRange'] - df['MinOfUpperTRange']
    df['lower_temp_range'] = df['MaxOfLowerTRange'] - df['MinOfLowerTRange']
    df['avg_temp'] = (df['AverageOfUpperTRange'] + df['AverageOfLowerTRange']) / 2
    
    # Fruit and pollination features
    df['fruit_efficiency'] = df['fruitset'] * df['fruitmass']
    df['seed_density'] = df['seeds'] / (df['fruitmass'] + 1)
    df['clone_fruit'] = df['clonesize'] * df['fruitset']
    
    # Weather interactions
    df['rain_temp_effect'] = df['RainingDays'] * df['avg_temp']
    df['bee_weather'] = df['total_bees'] * df['avg_temp'] * (1 / (df['RainingDays'] + 1))
    
    # Log transforms for skewed features
    for col in ['clonesize', 'fruitmass', 'seeds']:
        df[f'log_{col}'] = np.log1p(df[col])
    
    # Squared terms for key variables
    for col in ['fruitset', 'seeds', 'fruit_efficiency']:
        df[f'{col}_squared'] = df[col] ** 2
    
    return df

def train_models(X_train, y_train, X_val, y_val):
    models = {
        'xgb': xgb.XGBRegressor(
            n_estimators=3000,
            learning_rate=0.003,
            max_depth=6,
            subsample=0.85,
            colsample_bytree=0.85,
            min_child_weight=5,
            gamma=0.1,
            reg_alpha=0.1,
            reg_lambda=1,
            objective='reg:absoluteerror',
            tree_method='hist',
            random_state=42
        ),
        'lgbm': LGBMRegressor(
            n_estimators=3000,
            learning_rate=0.003,
            num_leaves=64,
            subsample=0.85,
            colsample_bytree=0.85,
            min_child_samples=20,
            objective='mae',
            random_state=42,
            force_col_wise=True
        ),
        'rf': RandomForestRegressor(
            n_estimators=500,
            max_depth=15,
            min_samples_split=5,
            min_samples_leaf=4,
            max_features='sqrt',
            criterion='absolute_error',
            random_state=42,
            n_jobs=-1
        ),
        'gb': GradientBoostingRegressor(
            n_estimators=500,
            learning_rate=0.03,
            max_depth=6,
            min_samples_split=5,
            min_samples_leaf=4,
            subsample=0.8,
            loss='absolute_error',
            random_state=42
        )
    }
    
    val_predictions = {}
    trained_models = {}
    
    for name, model in models.items():
        print(f"\nTraining {name.upper()} model...")
        model.fit(X_train, y_train)
        val_pred = model.predict(X_val)
        val_predictions[name] = val_pred
        trained_models[name] = model
        
        val_score = mean_absolute_error(y_val, val_pred)
        print(f"{name.upper()} Validation MAE: {val_score:.4f}")
    
    return trained_models, val_predictions

def blend_predictions(val_predictions, y_val):
    base_weights = {
        'xgb': 0.35,
        'lgbm': 0.30,
        'rf': 0.15,
        'gb': 0.20
    }
    
    best_mae = float('inf')
    best_weights = base_weights.copy()
    
    for xgb_w in np.arange(0.33, 0.37, 0.002):
        for lgbm_w in np.arange(0.28, 0.32, 0.002):
            for rf_w in np.arange(0.13, 0.17, 0.002):
                gb_w = 1 - xgb_w - lgbm_w - rf_w
                if 0.18 <= gb_w <= 0.22:
                    weights = {
                        'xgb': xgb_w,
                        'lgbm': lgbm_w,
                        'rf': rf_w,
                        'gb': gb_w
                    }
                    
                    pred = np.zeros(len(y_val))
                    for model, weight in weights.items():
                        pred += val_predictions[model] * weight
                    
                    mae = mean_absolute_error(y_val, pred)
                    if mae < best_mae:
                        best_mae = mae
                        best_weights = weights.copy()
    
    print(f"\nBest weights found: {best_weights}")
    print(f"Best Validation MAE: {best_mae:.4f}")
    return best_weights

def main():
    print("Loading and preprocessing data...")
    train, test, all_data, test_ids = load_and_preprocess()
    
    print("Creating features...")
    all_data_featured = create_features(all_data)
    
    drop_cols = ['id', 'Row#']
    all_data_featured = all_data_featured.drop(drop_cols, axis=1)
    
    print("\nData shape after feature engineering:", all_data_featured.shape)
    
    # Split data for feature selection
    X = all_data_featured.iloc[:len(train)]
    X_test = all_data_featured.iloc[len(train):]
    y = train['yield']
    
    # Perform feature selection
    selected_features = select_features(X, y, importance_threshold=0.0004)
    
    # Filter features based on selection
    X = X[selected_features]
    X_test = X_test[selected_features]
    
    print(f"\nTraining with {len(selected_features)} selected features")
    
    X_train, X_val, y_train, y_val = train_test_split(
        X, y, 
        test_size=0.2, 
        random_state=42
    )
    
    print("Scaling features...")
    scaler = RobustScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_val_scaled = scaler.transform(X_val)
    X_test_scaled = scaler.transform(X_test)
    
    X_train_scaled = pd.DataFrame(X_train_scaled, columns=X_train.columns)
    X_val_scaled = pd.DataFrame(X_val_scaled, columns=X_val.columns)
    X_test_scaled = pd.DataFrame(X_test_scaled, columns=X_test.columns)
    
    print("\nTraining models...")
    trained_models, val_predictions = train_models(X_train_scaled, y_train, X_val_scaled, y_val)
    
    print("\nFinding optimal blend weights...")
    best_weights = blend_predictions(val_predictions, y_val)
    
    print("\nGenerating final predictions...")
    test_predictions = {}
    for name, model in trained_models.items():
        test_predictions[name] = model.predict(X_test_scaled)
    
    final_predictions = np.zeros(len(X_test))
    for model, weight in best_weights.items():
        final_predictions += test_predictions[model] * weight
    
    submission = pd.DataFrame({
        'id': test_ids,
        'yield': final_predictions
    })
    submission['id'] = submission['id'].astype(int)
    submission = submission.sort_values('id').reset_index(drop=True)
    
    submission.to_csv('submission.csv', index=False)
    print("\nFirst few rows of submission file:")
    print(submission.head())
    print("\nSubmission file shape:", submission.shape)
    print("Submission files created successfully!")

if __name__ == "__main__":
    main()

Loading and preprocessing data...
Creating features...

Data shape after feature engineering: (25000, 33)

Performing Random Forest feature selection...

Feature Importances:
fruitset_squared: 0.4505
fruitset: 0.4020
seeds: 0.0400
log_seeds: 0.0352
seeds_squared: 0.0311
fruit_efficiency: 0.0058
seed_density: 0.0055
fruit_efficiency_squared: 0.0052
bee_weather: 0.0040
clone_fruit: 0.0035
fruitmass: 0.0027
log_fruitmass: 0.0027
bee_ratio: 0.0018
rain_temp_effect: 0.0014
total_bees: 0.0011
wild_bees: 0.0009
andrena: 0.0008
osmia: 0.0007
AverageOfLowerTRange: 0.0005
lower_temp_range: 0.0005
avg_temp: 0.0005
MinOfLowerTRange: 0.0005
MaxOfUpperTRange: 0.0004
bumbles: 0.0004
AverageOfUpperTRange: 0.0004
upper_temp_range: 0.0004
MaxOfLowerTRange: 0.0004
MinOfUpperTRange: 0.0003
AverageRainingDays: 0.0003
RainingDays: 0.0002
honeybee: 0.0001
log_clonesize: 0.0001
clonesize: 0.0001

Selected 27 features out of 33
Selected features: ['fruitset_squared', 'fruitset', 'seeds', 'log_seeds', 'seeds_sq

## Weste Paper but usefull

In [45]:
# from sklearn.ensemble import RandomForestRegressor
# from sklearn.model_selection import train_test_split, cross_val_score
# from sklearn.preprocessing import StandardScaler
# from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
# import lightgbm as lgb
# import numpy as np
# import pandas as pd

# def select_best_features_using_importance(X, y, n_estimators=200, max_depth=5, random_state=42):
#     X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=random_state)
    
#     scaler = StandardScaler()
#     X_train_scaled = scaler.fit_transform(X_train)
#     X_test_scaled = scaler.transform(X_test)
    
#     # Fit RandomForest to find feature importance
#     rf_model = RandomForestRegressor(n_estimators=n_estimators, max_depth=max_depth, random_state=random_state)
#     rf_model.fit(X_train_scaled, y_train)
    
#     # Get feature importance
#     feature_importances = rf_model.feature_importances_
    
#     # Sort features by importance
#     feature_importance_df = pd.DataFrame({
#         'Feature': X.columns,
#         'Importance': feature_importances
#     }).sort_values(by='Importance', ascending=False)
    
#     print("Features sorted by importance:")
#     print(feature_importance_df)
    
#     # Select top features
#     selected_features = feature_importance_df['Feature'].head(5)  # Selecting top 5 features
#     return selected_features

# def train_model_with_selected_features_cv(X, y, selected_features, testData=None, idTest=None):
#     X_selected = X[selected_features]
    
#     # Split the data for early stopping
#     X_train, X_val, y_train, y_val = train_test_split(X_selected, y, test_size=0.2, random_state=42)
    
#     scaler = StandardScaler()
#     X_train_scaled = scaler.fit_transform(X_train)
#     X_val_scaled = scaler.transform(X_val)
    
#     # Create LightGBM datasets
#     train_data = lgb.Dataset(X_train_scaled, label=y_train)
#     val_data = lgb.Dataset(X_val_scaled, label=y_val, reference=train_data)
    
#     # LightGBM parameters
#     params = {
#         'objective': 'regression',
#         'metric': 'mae',
#         'boosting_type': 'gbdt',
#         'num_leaves': 31,
#         'learning_rate': 0.05,
#         'feature_fraction': 0.9,
#         'bagging_fraction': 0.8,
#         'bagging_freq': 5,
#         'verbose': -1
#     }
    
#     # Training callbacks for early stopping
#     callbacks = [
#         lgb.early_stopping(stopping_rounds=50),
#         lgb.log_evaluation(period=100)
#     ]
    
#     # Train with early stopping
#     model = lgb.train(
#         params,
#         train_data,
#         valid_sets=[train_data, val_data],
#         num_boost_round=1000,
#         callbacks=callbacks
#     )
    
#     # Cross-validation for performance estimation
#     cv_results = lgb.cv(
#         params,
#         lgb.Dataset(X_selected, label=y),
#         num_boost_round=1000,
#         nfold=5,
#         stratified=False,
#         metrics=['mae'],
#         callbacks=[
#             lgb.early_stopping(stopping_rounds=50),
#             lgb.log_evaluation(period=100)
#         ],
#         seed=42,
#         return_cvbooster=True
#     )
    
#     # Debug: print available keys in cv_results
#     print("\nAvailable keys in cv_results:", cv_results.keys())
    
#     try:
#         # Try different possible key formats
#         if 'valid mae-mean' in cv_results:
#             scores_key = 'valid mae-mean'
#             std_key = 'valid mae-stdv'
#         elif 'valid_0 mae-mean' in cv_results:
#             scores_key = 'valid_0 mae-mean'
#             std_key = 'valid_0 mae-stdv'
#         elif 'l1-mean' in cv_results:
#             scores_key = 'l1-mean'
#             std_key = 'l1-stdv'
#         else:
#             # If none of the expected keys are found, use the first available metric
#             metric_keys = [k for k in cv_results.keys() if k.endswith('-mean')]
#             scores_key = metric_keys[0] if metric_keys else None
#             std_key = scores_key.replace('-mean', '-stdv') if scores_key else None
        
#         if scores_key:
#             valid_scores = cv_results[scores_key]
#             best_iteration = np.argmin(valid_scores)
#             best_mae = valid_scores[best_iteration]
#             mae_stddev = cv_results[std_key][best_iteration] if std_key else 0
            
#             print(f"\nBest CV MAE: {best_mae:.4f} ± {mae_stddev:.4f}")
#             print(f"Best iteration: {best_iteration + 1}")
#         else:
#             print("\nWarning: Could not find valid metric keys in CV results")
            
#     except Exception as e:
#         print(f"\nError processing CV results: {str(e)}")
#         print("Continuing with trained model...")
    
#     # Predict on test data if provided
#     if testData is not None:
#         X_test_scaled = scaler.transform(testData[selected_features])
#         test_predictions = model.predict(X_test_scaled)
        
#         output = pd.DataFrame({
#             'id': idTest,
#             'yield': test_predictions
#         })
        
#         output_file = 'submission.csv'
#         output.to_csv(output_file, index=False)
#         print(f"\nPredictions saved to {output_file}")
        
#         return model, output
    
#     return model

# # Example usage
# X = trainData[features_to_use]  # All features initially
# y = trainData['yield']

# # Step 1: Select best features using feature importance
# selected_features = select_best_features_using_importance(X, y)

# # Step 2: Train LightGBM model with early stopping
# model, predictions = train_model_with_selected_features_cv(
#     X, y, selected_features, 
#     testData=testData, 
#     idTest=testData['id']
# )

In [46]:
# from sklearn.ensemble import RandomForestRegressor
# from sklearn.model_selection import train_test_split, KFold
# from sklearn.preprocessing import StandardScaler
# from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
# import lightgbm as lgb
# import numpy as np
# import pandas as pd
# import matplotlib.pyplot as plt
# from typing import List, Tuple, Dict, Optional
# import warnings
# warnings.filterwarnings('ignore')

# def plot_feature_importance(importance_df: pd.DataFrame, title: str = "Feature Importance") -> None:
#     """Plot feature importance using a horizontal bar chart."""
#     plt.figure(figsize=(10, 6))
#     plt.barh(importance_df['Feature'][:10], importance_df['Importance'][:10])
#     plt.title(title)
#     plt.xlabel('Importance')
#     plt.ylabel('Features')
#     plt.tight_layout()
#     plt.show()

# def select_best_features_using_importance(
#     X: pd.DataFrame, 
#     y: pd.Series, 
#     n_features: int = 5,
#     n_estimators: int = 200, 
#     max_depth: int = 5, 
#     random_state: int = 42
# ) -> List[str]:
#     """
#     Select the best features using Random Forest importance.
    
#     Args:
#         X: Feature DataFrame
#         y: Target Series
#         n_features: Number of top features to select
#         n_estimators: Number of trees in Random Forest
#         max_depth: Maximum depth of trees
#         random_state: Random seed
        
#     Returns:
#         List of selected feature names
#     """
#     try:
#         X_train, _, y_train, _ = train_test_split(X, y, test_size=0.2, random_state=random_state)
        
#         scaler = StandardScaler()
#         X_train_scaled = scaler.fit_transform(X_train)
        
#         rf_model = RandomForestRegressor(
#             n_estimators=n_estimators, 
#             max_depth=max_depth, 
#             random_state=random_state,
#             n_jobs=-1
#         )
#         rf_model.fit(X_train_scaled, y_train)
        
#         feature_importance_df = pd.DataFrame({
#             'Feature': X.columns,
#             'Importance': rf_model.feature_importances_
#         }).sort_values(by='Importance', ascending=False)
        
#         print("\nTop 10 Features by Importance:")
#         print(feature_importance_df.head(10))
        
#         plot_feature_importance(feature_importance_df)
        
#         return feature_importance_df['Feature'].head(n_features).tolist()
        
#     except Exception as e:
#         print(f"Error in feature selection: {str(e)}")
#         raise

# def get_default_params() -> Dict:
#     """Get default LightGBM parameters."""
#     return {
#         'objective': 'regression',
#         'metric': 'mae',
#         'boosting_type': 'gbdt',
#         'num_leaves': 31,
#         'learning_rate': 0.05,
#         'feature_fraction': 0.9,
#         'bagging_fraction': 0.8,
#         'bagging_freq': 5,
#         'verbose': -1,
#         'n_jobs': -1,
#         'min_child_samples': 20,
#         'max_depth': -1  # Let LightGBM optimize the depth
#     }

# def train_model_with_selected_features_cv(
#     X: pd.DataFrame, 
#     y: pd.Series, 
#     selected_features: List[str],
#     testData: Optional[pd.DataFrame] = None,
#     idTest: Optional[pd.Series] = None,
#     n_folds: int = 5,
#     params: Optional[Dict] = None,
#     random_state: int = 42
# ) -> Tuple:
#     """
#     Train LightGBM model with cross-validation and early stopping.
    
#     Args:
#         X: Feature DataFrame
#         y: Target Series
#         selected_features: List of features to use
#         testData: Test DataFrame
#         idTest: Test IDs
#         n_folds: Number of CV folds
#         params: LightGBM parameters
#         random_state: Random seed
        
#     Returns:
#         Tuple of (model, predictions)
#     """
#     try:
#         X_selected = X[selected_features]
#         params = params or get_default_params()
        
#         # Initialize metrics storage
#         oof_predictions = np.zeros(len(X_selected))
#         feature_importance = np.zeros(len(selected_features))
#         scores = []
        
#         # Prepare KFold
#         kf = KFold(n_splits=n_folds, shuffle=True, random_state=random_state)
        
#         print(f"\nStarting {n_folds}-fold cross-validation...")
        
#         for fold, (train_idx, val_idx) in enumerate(kf.split(X_selected), 1):
#             print(f"\nFold {fold}/{n_folds}")
            
#             # Split data
#             X_train = X_selected.iloc[train_idx]
#             y_train = y.iloc[train_idx]
#             X_val = X_selected.iloc[val_idx]
#             y_val = y.iloc[val_idx]
            
#             # Scale features
#             scaler = StandardScaler()
#             X_train_scaled = scaler.fit_transform(X_train)
#             X_val_scaled = scaler.transform(X_val)
            
#             # Create datasets
#             train_data = lgb.Dataset(X_train_scaled, label=y_train)
#             val_data = lgb.Dataset(X_val_scaled, label=y_val, reference=train_data)
            
#             # Train model
#             model = lgb.train(
#                 params,
#                 train_data,
#                 valid_sets=[train_data, val_data],
#                 num_boost_round=1000,
#                 callbacks=[
#                     lgb.early_stopping(stopping_rounds=50),
#                     lgb.log_evaluation(period=100)
#                 ]
#             )
            
#             # Make validation predictions
#             oof_predictions[val_idx] = model.predict(X_val_scaled)
            
#             # Accumulate feature importance
#             fold_importance = model.feature_importance(importance_type='gain')
#             feature_importance += fold_importance
            
#             # Calculate fold score
#             fold_score = mean_absolute_error(y_val, oof_predictions[val_idx])
#             scores.append(fold_score)
#             print(f"Fold {fold} MAE: {fold_score:.4f}")
        
#         # Calculate and print overall metrics
#         mean_score = np.mean(scores)
#         std_score = np.std(scores)
#         print(f"\nOverall CV MAE: {mean_score:.4f} ± {std_score:.4f}")
        
#         # Plot feature importance
#         importance_df = pd.DataFrame({
#             'Feature': selected_features,
#             'Importance': feature_importance / n_folds
#         }).sort_values(by='Importance', ascending=False)
        
#         plot_feature_importance(importance_df, "LightGBM Feature Importance")
        
#         # Train final model on full dataset
#         print("\nTraining final model on full dataset...")
#         scaler = StandardScaler()
#         X_scaled = scaler.fit_transform(X_selected)
        
#         final_train_data = lgb.Dataset(X_scaled, label=y)
#         final_model = lgb.train(
#             params,
#             final_train_data,
#             num_boost_round=model.best_iteration,
#             callbacks=[lgb.log_evaluation(period=100)]
#         )
        
#         # Make predictions on test data if provided
#         if testData is not None and idTest is not None:
#             print("\nMaking predictions on test data...")
#             X_test_scaled = scaler.transform(testData[selected_features])
#             test_predictions = final_model.predict(X_test_scaled)
            
#             output = pd.DataFrame({
#                 'id': idTest,
#                 'yield': test_predictions
#             })
            
#             output_file = 'submission.csv'
#             output.to_csv(output_file, index=False)
#             print(f"Predictions saved to {output_file}")
            
#             return final_model, output
        
#         return final_model, None
        
#     except Exception as e:
#         print(f"Error in model training: {str(e)}")
#         raise

# # Example usage
# def main():
#     try:
#         X = trainData[features_to_use]
#         y = trainData['yield']
        
#         print("Starting feature selection...")
#         selected_features = select_best_features_using_importance(X, y)
        
#         print("\nStarting model training...")
#         model, predictions = train_model_with_selected_features_cv(
#             X, y, selected_features, 
#             testData=testData, 
#             idTest=testData['id']
#         )
        
#     except Exception as e:
#         print(f"Error in main execution: {str(e)}")
#         raise

# if __name__ == "__main__":
#     main()

In [47]:
# from sklearn.ensemble import RandomForestRegressor
# from sklearn.model_selection import train_test_split, KFold
# from sklearn.preprocessing import StandardScaler
# from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
# import lightgbm as lgb
# import numpy as np
# import pandas as pd
# import matplotlib.pyplot as plt
# import seaborn as sns
# from typing import List, Tuple, Dict, Optional, Union, Any
# import logging
# import joblib
# from pathlib import Path
# from datetime import datetime
# import yaml
# from dataclasses import dataclass
# import warnings
# import os

# warnings.filterwarnings('ignore')

# # Configure logging
# logging.basicConfig(
#     level=logging.INFO,
#     format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
#     handlers=[
#         logging.FileHandler('ml_pipeline.log'),
#         logging.StreamHandler()
#     ]
# )
# logger = logging.getLogger(__name__)

# @dataclass
# class ModelConfig:
#     """Data class for model configuration."""
#     n_features: int = 5
#     n_estimators: int = 200
#     max_depth: int = 5
#     n_folds: int = 5
#     random_state: int = 42
#     model_dir: str = "models"
    
#     @classmethod
#     def from_yaml(cls, yaml_path: str) -> 'ModelConfig':
#         """Load configuration from YAML file."""
#         with open(yaml_path, 'r') as f:
#             config_dict = yaml.safe_load(f)
#         return cls(**config_dict)

# class FeatureSelector:
#     """Class for feature selection operations."""
    
#     def __init__(self, config: ModelConfig):
#         self.config = config
#         self.feature_importance_df: Optional[pd.DataFrame] = None
        
#     def plot_feature_importance(self, importance_df: pd.DataFrame, title: str = "Feature Importance") -> None:
#         """Enhanced feature importance visualization."""
#         plt.figure(figsize=(12, 8))
#         sns.barplot(
#             data=importance_df.head(10),
#             x='Importance',
#             y='Feature',
#             palette='viridis'
#         )
#         plt.title(title, fontsize=14, pad=20)
#         plt.xlabel('Importance Score', fontsize=12)
#         plt.ylabel('Features', fontsize=12)
#         plt.tight_layout()
        
#         # Save plot
#         timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
#         plt.savefig(f'feature_importance_{timestamp}.png', dpi=300, bbox_inches='tight')
#         plt.close()

#     def select_features(
#         self,
#         X: pd.DataFrame,
#         y: pd.Series,
#         correlation_threshold: float = 0.95
#     ) -> List[str]:
#         """
#         Enhanced feature selection using both importance and correlation analysis.
#         """
#         try:
#             logger.info("Starting feature selection process...")
            
#             # Remove highly correlated features
#             correlation_matrix = X.corr().abs()
#             upper = correlation_matrix.where(
#                 np.triu(np.ones(correlation_matrix.shape), k=1).astype(bool)
#             )
#             to_drop = [
#                 column for column in upper.columns 
#                 if any(upper[column] > correlation_threshold)
#             ]
#             X_uncorrelated = X.drop(columns=to_drop)
#             logger.info(f"Removed {len(to_drop)} highly correlated features")
            
#             # Split and scale data
#             X_train, X_val, y_train, y_val = train_test_split(
#                 X_uncorrelated,
#                 y,
#                 test_size=0.2,
#                 random_state=self.config.random_state
#             )
            
#             scaler = StandardScaler()
#             X_train_scaled = scaler.fit_transform(X_train)
#             X_val_scaled = scaler.transform(X_val)
            
#             # Train Random Forest for feature importance
#             rf_model = RandomForestRegressor(
#                 n_estimators=self.config.n_estimators,
#                 max_depth=self.config.max_depth,
#                 random_state=self.config.random_state,
#                 n_jobs=-1
#             )
#             rf_model.fit(X_train_scaled, y_train)
            
#             # Calculate feature importance
#             self.feature_importance_df = pd.DataFrame({
#                 'Feature': X_uncorrelated.columns,
#                 'Importance': rf_model.feature_importances_,
#                 'Cumulative_Importance': np.cumsum(rf_model.feature_importances_)
#             }).sort_values(by='Importance', ascending=False)
            
#             logger.info("\nTop 10 Features by Importance:")
#             logger.info(self.feature_importance_df.head(10))
            
#             self.plot_feature_importance(self.feature_importance_df)
            
#             # Select features that explain 95% of the variance
#             selected_features = self.feature_importance_df[
#                 self.feature_importance_df['Cumulative_Importance'] <= 0.95
#             ]['Feature'].tolist()
            
#             return selected_features[:self.config.n_features]
            
#         except Exception as e:
#             logger.error(f"Error in feature selection: {str(e)}")
#             raise

# class ModelTrainer:
#     """Class for model training and evaluation."""
    
#     def __init__(self, config: ModelConfig):
#         self.config = config
#         self._ensure_model_dir()
        
#     def _ensure_model_dir(self) -> None:
#         """Ensure model directory exists."""
#         Path(self.config.model_dir).mkdir(parents=True, exist_ok=True)
        
#     def _get_default_params(self) -> Dict:
#         """Get enhanced LightGBM parameters."""
#         return {
#             'objective': 'regression',
#             'metric': ['mae', 'rmse'],
#             'boosting_type': 'gbdt',
#             'num_leaves': 31,
#             'learning_rate': 0.05,
#             'feature_fraction': 0.9,
#             'bagging_fraction': 0.8,
#             'bagging_freq': 5,
#             'verbose': -1,
#             'n_jobs': -1,
#             'min_child_samples': 20,
#             'max_depth': -1,
#             'reg_alpha': 0.1,
#             'reg_lambda': 0.1,
#             'min_split_gain': 0.01
#         }
    
#     def train_and_evaluate(
#         self,
#         X: pd.DataFrame,
#         y: pd.Series,
#         selected_features: List[str],
#         test_data: Optional[pd.DataFrame] = None,
#         test_ids: Optional[pd.Series] = None,
#         params: Optional[Dict] = None
#     ) -> Tuple[Any, Optional[pd.DataFrame], Dict[str, float]]:
#         """
#         Enhanced training with cross-validation and comprehensive evaluation.
#         """
#         try:
#             X_selected = X[selected_features]
#             params = params or self._get_default_params()
            
#             # Initialize arrays for predictions and importance
#             oof_predictions = np.zeros(len(X_selected))
#             feature_importance = np.zeros(len(selected_features))
#             metrics = {
#                 'mae_scores': [],
#                 'rmse_scores': [],
#                 'r2_scores': []
#             }
            
#             # Prepare KFold
#             kf = KFold(
#                 n_splits=self.config.n_folds,
#                 shuffle=True,
#                 random_state=self.config.random_state
#             )
            
#             logger.info(f"\nStarting {self.config.n_folds}-fold cross-validation...")
            
#             # Cross-validation loop
#             for fold, (train_idx, val_idx) in enumerate(kf.split(X_selected), 1):
#                 logger.info(f"\nFold {fold}/{self.config.n_folds}")
                
#                 # Split and scale data
#                 X_train = X_selected.iloc[train_idx]
#                 y_train = y.iloc[train_idx]
#                 X_val = X_selected.iloc[val_idx]
#                 y_val = y.iloc[val_idx]
                
#                 scaler = StandardScaler()
#                 X_train_scaled = scaler.fit_transform(X_train)
#                 X_val_scaled = scaler.transform(X_val)
                
#                 # Create datasets
#                 train_data = lgb.Dataset(X_train_scaled, label=y_train)
#                 val_data = lgb.Dataset(X_val_scaled, label=y_val, reference=train_data)
                
#                 # Train model
#                 model = lgb.train(
#                     params,
#                     train_data,
#                     valid_sets=[train_data, val_data],
#                     num_boost_round=1000,
#                     callbacks=[
#                         lgb.early_stopping(stopping_rounds=50),
#                         lgb.log_evaluation(period=100)
#                     ]
#                 )
                
#                 # Make predictions
#                 fold_preds = model.predict(X_val_scaled)
#                 oof_predictions[val_idx] = fold_preds
                
#                 # Calculate metrics
#                 metrics['mae_scores'].append(
#                     mean_absolute_error(y_val, fold_preds)
#                 )
#                 metrics['rmse_scores'].append(
#                     np.sqrt(mean_squared_error(y_val, fold_preds))
#                 )
#                 metrics['r2_scores'].append(
#                     r2_score(y_val, fold_preds)
#                 )
                
#                 # Update feature importance
#                 feature_importance += model.feature_importance(importance_type='gain')
                
#                 logger.info(
#                     f"Fold {fold} - MAE: {metrics['mae_scores'][-1]:.4f}, "
#                     f"RMSE: {metrics['rmse_scores'][-1]:.4f}, "
#                     f"R2: {metrics['r2_scores'][-1]:.4f}"
#                 )
            
#             # Calculate and log overall metrics
#             final_metrics = {
#                 'mean_mae': np.mean(metrics['mae_scores']),
#                 'std_mae': np.std(metrics['mae_scores']),
#                 'mean_rmse': np.mean(metrics['rmse_scores']),
#                 'std_rmse': np.std(metrics['rmse_scores']),
#                 'mean_r2': np.mean(metrics['r2_scores']),
#                 'std_r2': np.std(metrics['r2_scores'])
#             }
            
#             logger.info(f"\nFinal Model Metrics:\n{final_metrics}")
            
#             # Save final model
#             timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
#             final_model_path = os.path.join(self.config.model_dir, f'model_{timestamp}.pkl')
#             joblib.dump(model, final_model_path)
#             logger.info(f"Model saved at: {final_model_path}")
            
#             # Make predictions on test data if available
#             predictions_df = None
#             if test_data is not None and test_ids is not None:
#                 logger.info("\nMaking predictions on test data...")
#                 X_test_scaled = scaler.transform(test_data[selected_features])
#                 test_predictions = model.predict(X_test_scaled)

#                 predictions_df = pd.DataFrame({
#                     'id': test_ids,
#                     'yield': test_predictions
#                 })

#                 # Ensure output directory exists
#                 output_dir = 'output_model'
#                 if not os.path.exists(output_dir):
#                     os.makedirs(output_dir)

#                 # Save predictions to CSV file
#                 output_file = os.path.join(output_dir, f'predictions_{timestamp}.csv')
#                 predictions_df.to_csv(output_file, index=False)
#                 logger.info(f"Predictions saved to {output_file}")
                
#             return model, predictions_df, final_metrics
        
#         except Exception as e:
#             logger.error(f"Error during training and evaluation: {str(e)}")
#             raise

# class MLPipeline:
#     """Main pipeline class."""
    
#     def __init__(self, config: ModelConfig):
#         self.config = config
#         self.feature_selector = FeatureSelector(config)
#         self.model_trainer = ModelTrainer(config)
        
#     def run(
#         self,
#         train_data: pd.DataFrame,
#         target_column: str,
#         test_data: Optional[pd.DataFrame] = None,
#         feature_columns: Optional[List[str]] = None
#     ) -> Tuple[Any, Optional[pd.DataFrame], Dict[str, float]]:
#         """
#         Enhanced pipeline execution process.
#         """
#         try:
#             # Feature selection
#             X = train_data.drop(columns=[target_column])
#             y = train_data[target_column]
#             selected_features = (
#                 feature_columns if feature_columns 
#                 else self.feature_selector.select_features(X, y)
#             )
            
#             logger.info(f"Selected Features: {selected_features}")
            
#             # Train and evaluate the model
#             model, predictions, metrics = self.model_trainer.train_and_evaluate(
#                 X,
#                 y,
#                 selected_features,
#                 test_data=test_data,
#                 test_ids=test_data.index if test_data is not None else None
#             )
            
#             return model, predictions, metrics
        
#         except Exception as e:
#             logger.error(f"Error in pipeline run: {str(e)}")
#             raise

# def main():
#     # Load configuration
#     config = ModelConfig.from_yaml("config.yaml")
    
#     # Load data
#     trainData = pd.read_csv('train.csv')
#     testData = pd.read_csv('test.csv')  # If available
    
#     # Run pipeline
#     pipeline = MLPipeline(config)
#     model, predictions, metrics = pipeline.run(
#         train_data=trainData,
#         target_column='yield',
#         test_data=testData
#     )

# if __name__ == "__main__":
#     main()


In [48]:
# %pip uninstall lightgbm -y
# %pip install lightgbm

In [49]:
# import pandas as pd
# import numpy as np
# from sklearn.ensemble import RandomForestRegressor
# from sklearn.model_selection import train_test_split
# from sklearn.preprocessing import StandardScaler
# from sklearn.metrics import mean_absolute_error
# from lightgbm import LGBMRegressor

# # Step 1: Select best features using Random Forest Feature Importance
# def select_best_features_using_importance(X, y, n_estimators=200, max_depth=5, random_state=42):
#     X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=random_state)
    
#     # Normalize features
#     scaler = StandardScaler()
#     X_train_scaled = scaler.fit_transform(X_train)
#     X_test_scaled = scaler.transform(X_test)
    
#     # Fit RandomForest to find feature importance
#     rf_model = RandomForestRegressor(n_estimators=n_estimators, max_depth=max_depth, random_state=random_state)
#     rf_model.fit(X_train_scaled, y_train)
    
#     # Get feature importance
#     feature_importances = rf_model.feature_importances_
    
#     # Sort features by importance
#     feature_importance_df = pd.DataFrame({
#         'Feature': X.columns,
#         'Importance': feature_importances
#     }).sort_values(by='Importance', ascending=False)
    
#     print("Features sorted by importance:")
#     print(feature_importance_df)
    
#     # Select top features
#     selected_features = feature_importance_df['Feature'].head(5)  # Selecting top 5 features
#     return selected_features

# # Step 2: Train a model using selected features with Cross-Validation
# def train_model_with_selected_features_cv(X, y, selected_features, testData=None, idTest=None):
#     # Prepare the selected features dataframe
#     X_selected = X[selected_features]

#     # Normalize the features
#     scaler = StandardScaler()
#     X_scaled = scaler.fit_transform(X_selected)

#     # Split the dataset into training and validation sets
#     X_train, X_val, y_train, y_val = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

#     # LightGBM Model
#     model = LGBMRegressor(n_estimators=1000, learning_rate=0.09, max_depth=5, random_state=42)

#     # Fit the model with early stopping
#     model.fit(X_train, y_train, 
#               eval_set=[(X_val, y_val)], 
#               eval_metric='mae', 
#               early_stopping_rounds=50, 
#               verbose=100)

#     # Validation Predictions
#     val_predictions = model.predict(X_val)
#     mae = mean_absolute_error(y_val, val_predictions)
#     print(f"\nValidation MAE: {mae:.4f}")

#     # Fit the model on the entire dataset if test data is provided
#     model.fit(X_scaled, y)

#     # Predicting on test data
#     if testData is not None:
#         X_test_scaled = scaler.transform(testData[selected_features])
#         testPredictions = model.predict(X_test_scaled)
        
#         output = pd.DataFrame({
#             'id': idTest,
#             'yield': testPredictions
#         })
        
#         output_file = 'submission.csv'
#         output.to_csv(output_file, index=False)
#         print(f"Predictions saved to {output_file}")

# # Example usage with your datasets
# # Ensure you have your trainData and testData DataFrame populated with data
# # trainData = pd.read_csv('path_to_train_data.csv')  # Uncomment and specify path
# # testData = pd.read_csv('path_to_test_data.csv')    # Uncomment and specify path

# # Specify features you want to use
# features_to_use = trainData.columns.drop('yield')  # Adjust according to your dataset
# features_to_use = features_to_use.drop('id')  # Adjust according to your dataset
# features_to_use = features_to_use.drop('Row#')  # Adjust according to your dataset
# X = trainData[features_to_use]  # All features initially
# y = trainData['yield']

# # Step 1: Select best features using feature importance
# selected_features = select_best_features_using_importance(X, y)

# # Step 2: Train a model with cross-validation using those selected features
# train_model_with_selected_features_cv(
#     X, 
#     y, 
#     selected_features, 
#     testData=testData, 
#     idTest=testData['id']  # Ensure 'id' column exists in testData
# )

In [ ]:
testData.columns

In [51]:
# import pandas as pd
# import lightgbm as lgb
# from sklearn.model_selection import train_test_split, GridSearchCV
# from sklearn.metrics import mean_squared_error, mean_absolute_error

# # Load the data
# train_data = pd.read_csv('train.csv')
# test_data = pd.read_csv('test.csv')

# # Separate features and target
# X = train_data.drop(columns=['yield', 'id', 'Row#'])
# y = train_data['yield']

# # Split data into training and validation sets
# X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# # Create LightGBM dataset structures
# train_set = lgb.Dataset(X_train, label=y_train)
# val_set = lgb.Dataset(X_val, label=y_val)

# # Set model parameters
# params = {
#     'objective': 'regression',
#     'metric': 'rmse',  # Root mean squared error
#     'boosting_type': 'gbdt',
#     'verbose': -1
# }

# # Define the parameter grid
# param_grid = {
#     'learning_rate': [0.01, 0.05, 0.1],
#     'num_leaves': [20, 31, 50],
#     'max_depth': [-1, 10, 20],
#     'feature_fraction': [0.8, 0.9, 1.0],
#     'bagging_fraction': [0.8, 0.9, 1.0],
#     'bagging_freq': [1, 5, 10],
# }

# # Initialize the model
# lgb_model = lgb.LGBMRegressor(**params)

# # Use GridSearchCV for hyperparameter tuning
# grid_search = GridSearchCV(lgb_model, param_grid, scoring='neg_mean_squared_error', cv=3, verbose=2)
# grid_search.fit(X_train, y_train)

# # Best parameters
# print("Best parameters found: ", grid_search.best_params_)

# # Train the model with the best parameters
# best_model = grid_search.best_estimator_

# # Validate the model
# y_pred_val = best_model.predict(X_val)
# val_rmse = mean_squared_error(y_val, y_pred_val, squared=False)
# val_mae = mean_absolute_error(y_val, y_pred_val)

# print(f'Validation RMSE: {val_rmse}')
# print(f'Validation MAE: {val_mae}')

# # Predict on the test set
# X_test = test_data.drop(columns=['id', 'Row#'])
# y_pred_test = best_model.predict(X_test)

# # Create a DataFrame for the output
# output = pd.DataFrame({
#     'id': test_data['id'],
#     'prediction': y_pred_test
# })

# # Save the predictions to a CSV file
# output.to_csv('test_predictions.csv', index=False)
# print("Predictions saved to 'test_predictions.csv'.")


In [52]:
# import lightgbm as lgb
# from lightgbm import early_stopping
# from sklearn.model_selection import train_test_split
# from sklearn.preprocessing import StandardScaler
# from sklearn.metrics import mean_absolute_error
# import pandas as pd
# import numpy as np

# # Step 1: Train a model using all features with LightGBM and early stopping
# def train_model_with_all_features_cv(X, y, testData=None, idTest=None, model_type='lgbm'):
#     scaler = StandardScaler()
#     X_scaled = scaler.fit_transform(X)  # Standardize the features

#     # Select model type
#     if model_type == 'lgbm':
#         model = lgb.LGBMRegressor(n_estimators=1000, learning_rate=0.05, max_depth=5, random_state=42)
#     else:
#         raise ValueError("Unsupported model type")

#     # Split into train and validation set for early stopping
#     X_train, X_val, y_train, y_val = train_test_split(X_scaled, y, test_size=0.2, random_state=42)
    
#     # Fit model with early stopping using callbacks
#     model.fit(X_train, y_train, 
#               eval_set=[(X_val, y_val)], 
#               callbacks=[early_stopping(stopping_rounds=50, verbose=100)],  # Stops if no improvement over 50 rounds
#               eval_metric='mae')  # Monitor mean absolute error
    
#     # Predict on validation set
#     val_predictions = model.predict(X_val)
    
#     # Evaluate performance on validation set
#     val_mae = mean_absolute_error(y_val, val_predictions)
#     print(f"\nValidation MAE: {val_mae:.4f}")
    
#     # Fit the model on the entire dataset (without validation split) after early stopping
#     model.fit(X_scaled, y)
    
#     # Predict on testData (optional)
#     if testData is not None:
#         X_test_scaled = scaler.transform(testData)
#         testPredictions = model.predict(X_test_scaled)
        
#         output = pd.DataFrame({
#             'id': idTest,
#             'yield': testPredictions
#         })
        
#         output_file = 'submission.csv'
#         output.to_csv(output_file, index=False)
#         print(f"Predictions saved to {output_file}")

# # Example usage
# X = trainData.drop(columns=['yield'])  # All features except the target variable
# y = trainData['yield']

# # Step 1: Train a model with cross-validation using all features and early stopping
# train_model_with_all_features_cv(X, y, testData=testData, idTest=testData['id'], model_type='lgbm')


In [ ]:
# from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor
# from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
# from sklearn.preprocessing import StandardScaler
# from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
# import numpy as np
# import pandas as pd

# def select_best_features_using_importance(X, y, n_estimators=200, max_depth=5, random_state=42):
#     X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=random_state)
    
#     scaler = StandardScaler()
#     X_train_scaled = scaler.fit_transform(X_train)
#     X_test_scaled = scaler.transform(X_test)
    
#     rf_model = RandomForestRegressor(n_estimators=n_estimators, max_depth=max_depth, random_state=random_state)
#     rf_model.fit(X_train_scaled, y_train)
    
#     feature_importances = rf_model.feature_importances_
    
#     feature_importance_df = pd.DataFrame({
#         'Feature': X.columns,
#         'Importance': feature_importances
#     }).sort_values(by='Importance', ascending=False)
    
#     print("Features sorted by importance:")
#     print(feature_importance_df)
    
#     selected_features = feature_importance_df['Feature'].head(5)
#     return selected_features

# def train_model_with_selected_features_cv(X, y, selected_features, testData=None, idTest=None, model_type='gradient_boosting'):
#     X_selected = X[selected_features]
    
#     scaler = StandardScaler()
#     X_scaled = scaler.fit_transform(X_selected)

#     if model_type == 'gradient_boosting':
#         # Define the parameter grid for GridSearchCV
#         param_grid = {
#             'n_estimators': [100, 120, 150],
#             'learning_rate': [0.05, 0.06, 0.09, 0.1],
#             'max_depth': [3, 4, 5, 6],
#             'min_samples_split': [2, 5, 7],
#             'min_samples_leaf': [1, 2, 4, 5],
#             'subsample': [0.5, 0.7, 0.8, 0.9, 1.0]
#         }
        
#         base_model = GradientBoostingRegressor(random_state=42)
#     else:
#         raise ValueError("Unsupported model type")

#     # Perform GridSearchCV
#     grid_search = GridSearchCV(
#         estimator=base_model,
#         param_grid=param_grid,
#         cv=5,
#         scoring='neg_mean_absolute_error',
#         n_jobs=-1,
#         verbose=2
#     )
    
#     # Fit GridSearchCV
#     grid_search.fit(X_scaled, y)
    
#     # Print best parameters and score
#     print("\nBest parameters found:")
#     print(grid_search.best_params_)
#     print(f"Best cross-validated MAE: {-grid_search.best_score_:.4f}")
    
#     # Use best model for predictions
#     best_model = grid_search.best_estimator_
    
#     # Perform 5-Fold Cross-Validation with best model
#     cv_scores = cross_val_score(best_model, X_scaled, y, cv=5, scoring='neg_mean_absolute_error')
#     print(f"\nFinal Cross-validated MAE (5-Fold): {np.mean(-cv_scores):.4f}")
    
#     # Fit the best model on the entire dataset
#     best_model.fit(X_scaled, y)
    
#     # Predict on testData if provided
#     if testData is not None:
#         X_test_scaled = scaler.transform(testData[selected_features])
#         testPredictions = best_model.predict(X_test_scaled)
        
#         output = pd.DataFrame({
#             'id': idTest,
#             'yield': testPredictions
#         })
        
#         output_file = 'submission.csv'
#         output.to_csv(output_file, index=False)
#         print(f"Predictions saved to {output_file}")
        
#     return best_model, grid_search

# # Example usage
# X = trainData[features_to_use]  # All features initially
# y = trainData['yield']

# # Step 1: Select best features using feature importance
# selected_features = select_best_features_using_importance(X, y)

# # Step 2: Train a model with GridSearchCV using those selected features
# best_model, grid_search = train_model_with_selected_features_cv(
#     X, y, selected_features, testData=testData, idTest=testData['id']
# )

# # Additional analysis of GridSearchCV results (optional)
# cv_results = pd.DataFrame(grid_search.cv_results_)
# print("\nTop 5 parameter combinations:")
# print(cv_results[['params', 'mean_test_score', 'rank_test_score']]
#       .sort_values('rank_test_score')
#       .head())

In [ ]:
# from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor
# from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
# from sklearn.preprocessing import StandardScaler
# from sklearn.metrics import mean_absolute_error
# import numpy as np
# import pandas as pd

# def select_best_features_using_importance(X, y, n_estimators=200, max_depth=5, random_state=42):
#     X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=random_state)
    
#     scaler = StandardScaler()
#     X_train_scaled = scaler.fit_transform(X_train)
    
#     rf_model = RandomForestRegressor(n_estimators=n_estimators, max_depth=max_depth, random_state=random_state)
#     rf_model.fit(X_train_scaled, y_train)
    
#     feature_importances = rf_model.feature_importances_
    
#     feature_importance_df = pd.DataFrame({
#         'Feature': X.columns,
#         'Importance': feature_importances
#     }).sort_values(by='Importance', ascending=False)
    
#     print("Features sorted by importance:")
#     print(feature_importance_df)
    
#     selected_features = feature_importance_df['Feature'].head(5)
#     return selected_features

# def train_model_with_selected_features_cv(X, y, selected_features, testData=None, idTest=None):
#     X_selected = X[selected_features]
    
#     scaler = StandardScaler()
#     X_scaled = scaler.fit_transform(X_selected)

#     # Split the data into training and validation sets
#     X_train, X_val, y_train, y_val = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

#     # Define the model
#     model = GradientBoostingRegressor(n_estimators=120, learning_rate=0.09, max_depth=5, random_state=42)

#     # Fit the model with early stopping
#     model.fit(X_train, y_train)

#     # Early stopping
#     best_iter = 0
#     best_score = float('inf')
#     for i, y_pred in enumerate(model.staged_predict(X_val)):
#         val_score = mean_absolute_error(y_val, y_pred)
#         if val_score < best_score:
#             best_score = val_score
#             best_iter = i + 1  # +1 because staged_predict starts from 0
#         else:
#             break  # Stop if the validation score doesn't improve

#     print(f"Best iteration: {best_iter}, Best MAE: {best_score:.4f}")

#     # Fit the model again with the best iteration
#     model = GradientBoostingRegressor(n_estimators=best_iter, learning_rate=0.09, max_depth=5, random_state=42)
#     model.fit(X_scaled, y)

#     # Predict on testData if provided
#     if testData is not None:
#         X_test_scaled = scaler.transform(testData[selected_features])
#         testPredictions = model.predict(X_test_scaled)
        
#         output = pd.DataFrame({
#             'id': idTest,
#             'yield': testPredictions
#         })
        
#         output_file = 'submission_ES.csv'
#         output.to_csv(output_file, index=False)
#         print(f"Predictions saved to {output_file}")

# # Example usage
# X = trainData[features_to_use]  # All features initially
# y = trainData['yield']

# # Step 1: Select best features using feature importance
# selected_features = select_best_features_using_importance(X, y)

# # Step 2: Train a model with early stopping using those selected features
# train_model_with_selected_features_cv(X, y, selected_features, testData=testData, idTest=testData['id'])

In [3]:
import pandas as pd
import numpy as np
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error
import lightgbm as lgb
import xgboost as xgb
from sklearn.ensemble import GradientBoostingRegressor

In [8]:
# Load the data
train_data = pd.read_csv('train.csv')
test_data = pd.read_csv('test.csv')

# Separate features and target variable from training data
X_train = train_data.drop(columns=['id', 'Row#', 'yield'])  # Drop non-feature columns
y_train = train_data['yield']
X_test = test_data.drop(columns=['id', 'Row#'])  # Drop non-feature columns

In [9]:
# def evaluate_model(model, X, y):
#     kf = KFold(n_splits=5, shuffle=True, random_state=42)
#     scores = []
    
#     for train_index, val_index in kf.split(X):
#         X_train_fold, X_val_fold = X.iloc[train_index], X.iloc[val_index]
#         y_train_fold, y_val_fold = y.iloc[train_index], y.iloc[val_index]
        
#         model.fit(X_train_fold, y_train_fold, 
#                   eval_set=[(X_val_fold, y_val_fold)], 
#                   early_stopping_rounds=50, 
#                   verbose=False)
        
#         y_pred = model.predict(X_val_fold)
#         score = mean_squared_error(y_val_fold, y_pred, squared=False)  # RMSE
#         scores.append(score)
    
#     return np.mean(scores), np.std(scores)

In [ ]:
# # LightGBM
# lgb_model = lgb.LGBMRegressor()
# lgb_mean_score, lgb_std_score = evaluate_model(lgb_model, X_train, y_train)
# print(f'LightGBM: Mean RMSE = {lgb_mean_score:.4f}, Std = {lgb_std_score:.4f}')

# # XGBoost
# xgb_model = xgb.XGBRegressor(eval_metric='rmse')
# xgb_mean_score, xgb_std_score = evaluate_model(xgb_model, X_train, y_train)
# print(f'XGBoost: Mean RMSE = {xgb_mean_score:.4f}, Std = {xgb_std_score:.4f}')

# # Gradient Boosting
# gb_model = GradientBoostingRegressor()
# gb_mean_score, gb_std_score = evaluate_model(gb_model, X_train, y_train)
# print(f'Gradient Boosting: Mean RMSE = {gb_mean_score:.4f}, Std = {gb_std_score:.4f}')

In [ ]:
# # Choose the best model based on the evaluation
# best_model = lgb_model  # or xgb_model, or gb_model based on your evaluation

# # Fit the best model on the entire training data
# best_model.fit(X_train, y_train)

# # Make predictions on the test set
# predictions = best_model.predict(X_test)

# # Create a submission DataFrame
# submission = pd.DataFrame({
#     'id': test_data['id'],
#     'predicted_yield': predictions
# })

# # Save predictions to a CSV file
# submission.to_csv('submission.csv', index=False)

In [ ]:
# import pandas as pd
# import numpy as np
# from sklearn.model_selection import KFold
# from sklearn.metrics import mean_squared_error, mean_absolute_error
# import lightgbm as lgb
# import xgboost as xgb
# from sklearn.ensemble import GradientBoostingRegressor

# # Load the data
# train_data = pd.read_csv('train.csv')
# test_data = pd.read_csv('test.csv')

# # Separate features and target variable from training data
# X_train = train_data.drop(columns=['id', 'Row#', 'yield'])
# y_train = train_data['yield']
# X_test = test_data.drop(columns=['id', 'Row#'])

# # Define separate evaluation functions for each model type
# def evaluate_lgb(X, y):
#     kf = KFold(n_splits=5, shuffle=True, random_state=42)
#     rmse_scores = []
#     mae_scores = []
    
#     for train_index, val_index in kf.split(X):
#         X_train_fold, X_val_fold = X.iloc[train_index], X.iloc[val_index]
#         y_train_fold, y_val_fold = y.iloc[train_index], y.iloc[val_index]
        
#         model = lgb.LGBMRegressor(n_estimators=1000)
#         model.fit(X_train_fold, y_train_fold,
#                  eval_set=[(X_val_fold, y_val_fold)],
#                  callbacks=[lgb.early_stopping(50)])
        
#         y_pred = model.predict(X_val_fold)
#         rmse = mean_squared_error(y_val_fold, y_pred, squared=False)
#         mae = mean_absolute_error(y_val_fold, y_pred)
#         rmse_scores.append(rmse)
#         mae_scores.append(mae)
    
#     return (np.mean(rmse_scores), np.std(rmse_scores), 
#             np.mean(mae_scores), np.std(mae_scores))

# def evaluate_xgb(X, y):
#     kf = KFold(n_splits=5, shuffle=True, random_state=42)
#     rmse_scores = []
#     mae_scores = []
    
#     for train_index, val_index in kf.split(X):
#         X_train_fold, X_val_fold = X.iloc[train_index], X.iloc[val_index]
#         y_train_fold, y_val_fold = y.iloc[train_index], y.iloc[val_index]
        
#         model = xgb.XGBRegressor(n_estimators=1000)
#         model.fit(X_train_fold, y_train_fold)  # Simple fit without early stopping
        
#         y_pred = model.predict(X_val_fold)
#         rmse = mean_squared_error(y_val_fold, y_pred, squared=False)
#         mae = mean_absolute_error(y_val_fold, y_pred)
#         rmse_scores.append(rmse)
#         mae_scores.append(mae)
    
#     return (np.mean(rmse_scores), np.std(rmse_scores), 
#             np.mean(mae_scores), np.std(mae_scores))

# def evaluate_gb(X, y):
#     kf = KFold(n_splits=5, shuffle=True, random_state=42)
#     rmse_scores = []
#     mae_scores = []
    
#     for train_index, val_index in kf.split(X):
#         X_train_fold, X_val_fold = X.iloc[train_index], X.iloc[val_index]
#         y_train_fold, y_val_fold = y.iloc[train_index], y.iloc[val_index]
        
#         model = GradientBoostingRegressor(n_estimators=100)
#         model.fit(X_train_fold, y_train_fold)
        
#         y_pred = model.predict(X_val_fold)
#         rmse = mean_squared_error(y_val_fold, y_pred, squared=False)
#         mae = mean_absolute_error(y_val_fold, y_pred)
#         rmse_scores.append(rmse)
#         mae_scores.append(mae)
    
#     return (np.mean(rmse_scores), np.std(rmse_scores), 
#             np.mean(mae_scores), np.std(mae_scores))

# # Evaluate models
# print("Evaluating LightGBM...")
# lgb_rmse_mean, lgb_rmse_std, lgb_mae_mean, lgb_mae_std = evaluate_lgb(X_train, y_train)
# print(f'LightGBM: RMSE = {lgb_rmse_mean:.4f} (±{lgb_rmse_std:.4f})')
# print(f'LightGBM: MAE = {lgb_mae_mean:.4f} (±{lgb_mae_std:.4f})')

# print("\nEvaluating XGBoost...")
# xgb_rmse_mean, xgb_rmse_std, xgb_mae_mean, xgb_mae_std = evaluate_xgb(X_train, y_train)
# print(f'XGBoost: RMSE = {xgb_rmse_mean:.4f} (±{xgb_rmse_std:.4f})')
# print(f'XGBoost: MAE = {xgb_mae_mean:.4f} (±{xgb_mae_std:.4f})')

# print("\nEvaluating Gradient Boosting...")
# gb_rmse_mean, gb_rmse_std, gb_mae_mean, gb_mae_std = evaluate_gb(X_train, y_train)
# print(f'Gradient Boosting: RMSE = {gb_rmse_mean:.4f} (±{gb_rmse_std:.4f})')
# print(f'Gradient Boosting: MAE = {gb_mae_mean:.4f} (±{gb_mae_std:.4f})')

# # Find the best model based on mean RMSE
# models_scores = {
#     'LightGBM': lgb_rmse_mean,
#     'XGBoost': xgb_rmse_mean,
#     'Gradient Boosting': gb_rmse_mean
# }

# best_model_name = min(models_scores, key=models_scores.get)
# print(f"\nBest model is {best_model_name} with RMSE: {models_scores[best_model_name]:.4f}")

# # Train the best model and make predictions
# if best_model_name == 'LightGBM':
#     final_model = lgb.LGBMRegressor(n_estimators=1000)
#     final_model.fit(X_train, y_train,
#                    eval_set=[(X_train.iloc[:100], y_train.iloc[:100])],
#                    callbacks=[lgb.early_stopping(50)])
# elif best_model_name == 'XGBoost':
#     final_model = xgb.XGBRegressor(n_estimators=1000)
#     final_model.fit(X_train, y_train)
# else:
#     final_model = GradientBoostingRegressor(n_estimators=100)
#     final_model.fit(X_train, y_train)

# # Make predictions on test set
# predictions = final_model.predict(X_test)

# # Create submission file
# submission = pd.DataFrame({
#     'id': test_data['id'],
#     'predicted_yield': predictions
# })
# submission.to_csv('submission.csv', index=False)

In [ ]:
# import pandas as pd
# import numpy as np
# from sklearn.model_selection import train_test_split
# from sklearn.metrics import mean_squared_error, mean_absolute_error
# import lightgbm as lgb
# import xgboost as xgb
# from sklearn.ensemble import GradientBoostingRegressor

# # Load the data
# train_data = pd.read_csv('train.csv')
# test_data = pd.read_csv('test.csv')

# # Separate features and target variable from training data
# X = train_data.drop(columns=['id', 'Row#', 'yield'])
# y = train_data['yield']
# X_test = test_data.drop(columns=['id', 'Row#'])

# # Split the data into training and validation sets (80% train, 20% validation)
# X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# # Define separate evaluation functions for each model type
# def evaluate_lgb(X_train, y_train, X_val, y_val):
#     model = lgb.LGBMRegressor(n_estimators=1000)
#     model.fit(X_train, y_train,
#               eval_set=[(X_val, y_val)],
#               callbacks=[lgb.early_stopping(50)])
    
#     y_pred = model.predict(X_val)
#     rmse = mean_squared_error(y_val, y_pred, squared=False)
#     mae = mean_absolute_error(y_val, y_pred)
    
#     return rmse, mae

# def evaluate_xgb(X_train, y_train, X_val, y_val):
#     model = xgb.XGBRegressor(n_estimators=1000)
#     eval_set = [(X_val, y_val)]
#     model.fit(X_train, y_train,
#               eval_set=eval_set,
#               verbose=False)
    
#     y_pred = model.predict(X_val)
#     rmse = mean_squared_error(y_val, y_pred, squared=False)
#     mae = mean_absolute_error(y_val, y_pred)
    
#     return rmse, mae

# def evaluate_gb(X_train, y_train, X_val, y_val):
#     model = GradientBoostingRegressor(n_estimators=100, validation_fraction=0.1, n_iter_no_change=50)
#     model.fit(X_train, y_train)
    
#     y_pred = model.predict(X_val)
#     rmse = mean_squared_error(y_val, y_pred, squared=False)
#     mae = mean_absolute_error(y_val, y_pred)
    
#     return rmse, mae

# # Evaluate models
# print("Evaluating LightGBM...")
# lgb_rmse, lgb_mae = evaluate_lgb(X_train, y_train, X_val, y_val)
# print(f'LightGBM: RMSE = {lgb_rmse:.4f}')
# print(f'LightGBM: MAE = {lgb_mae:.4f}')

# print("\nEvaluating XGBoost...")
# xgb_rmse, xgb_mae = evaluate_xgb(X_train, y_train, X_val, y_val)
# print(f'XGBoost: RMSE = {xgb_rmse:.4f}')
# print(f'XGBoost: MAE = {xgb_mae:.4f}')

# print("\nEvaluating Gradient Boosting...")
# gb_rmse, gb_mae = evaluate_gb(X_train, y_train, X_val, y_val)
# print(f'Gradient Boosting: RMSE = {gb_rmse:.4f}')
# print(f'Gradient Boosting: MAE = {gb_mae:.4f}')

# # Find the best model based on mean RMSE
# models_scores = {
#     'LightGBM': lgb_rmse,
#     'XGBoost': xgb_rmse,
#     'Gradient Boosting': gb_rmse
# }

# best_model_name = min(models_scores, key=models_scores.get)
# print(f"\nBest model is {best_model_name} with RMSE: {models_scores[best_model_name]:.4f}")

# # Train the best model and make predictions
# if best_model_name == 'LightGBM':
#     final_model = lgb.LGBMRegressor(n_estimators=1000)
#     final_model.fit(X, y,
#                    eval_set=[(X_val, y_val)],
#                    callbacks=[lgb.early_stopping(50)])
# elif best_model_name == 'XGBoost':
#     final_model = xgb.XGBRegressor(n_estimators=1000)
#     eval_set = [(X_val, y_val)]
#     final_model.fit(X, y,
#                    eval_set=eval_set,
#                    verbose=False)
# else:
#     final_model = GradientBoostingRegressor(n_estimators=100, validation_fraction=0.1, n_iter_no_change=50)
#     final_model.fit(X, y)

# # Make predictions on test set
# predictions = final_model.predict(X_test)

# # Create submission file
# submission = pd.DataFrame({
#     'id': test_data['id'],
#     'predicted_yield': predictions
# })
# submission.to_csv('submission.csv', index=False)

In [ ]:
# import pandas as pd
# import lightgbm as lgb
# from sklearn.model_selection import train_test_split, KFold, GridSearchCV
# from sklearn.metrics import mean_absolute_error, mean_squared_error
# from sklearn.preprocessing import StandardScaler
# import numpy as np

# # Load train and test datasets
# train = pd.read_csv('train.csv')
# test = pd.read_csv('test.csv')

# # Prepare the data (Feature Scaling)
# X = train.drop(columns=['id', 'Row#', 'yield'])  # Features for training
# y = train['yield']  # Target variable

# # Standardize features for better performance
# scaler = StandardScaler()
# X_scaled = scaler.fit_transform(X)

# # Convert test data features (excluding 'id' and 'Row#')
# X_test = test.drop(columns=['id', 'Row#'])
# X_test_scaled = scaler.transform(X_test)

# # Split the data into training and validation sets
# X_train, X_valid, y_train, y_valid = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# # Define a parameter grid for GridSearchCV
# param_grid = {
#     'learning_rate': [0.01, 0.05, 0.1],
#     'num_leaves': [31, 50, 70],
#     'min_data_in_leaf': [20, 40, 60],
#     'feature_fraction': [0.7, 0.8, 0.9],
#     'bagging_fraction': [0.7, 0.8, 0.9],
#     'lambda_l1': [0.0, 0.1, 0.5],
#     'lambda_l2': [0.0, 0.1, 0.5]
# }

# # Initialize the LightGBM estimator
# lgb_estimator = lgb.LGBMRegressor(boosting_type='gbdt', objective='regression', n_estimators=1000)

# # Perform GridSearchCV without early stopping
# grid_search = GridSearchCV(estimator=lgb_estimator, param_grid=param_grid, cv=5, scoring='neg_mean_absolute_error', verbose=1)

# # Fit the Grid Search (no early stopping here)
# grid_search.fit(X_scaled, y)

# # Best parameters from grid search
# best_params = grid_search.best_params_
# print(f"Best parameters: {best_params}")

# # Retrain the model using the best parameters and apply early stopping
# best_model = lgb.LGBMRegressor(**best_params, n_estimators=1000)

# best_model.fit(X_train, y_train,
#                eval_set=[(X_valid, y_valid)],
#                early_stopping_rounds=100,
#                verbose=100)

# # Cross-validation with the best parameters
# kf = KFold(n_splits=5, shuffle=True, random_state=42)
# mae_scores = []
# rmse_scores = []

# for train_index, val_index in kf.split(X_scaled):
#     X_train_fold, X_val_fold = X_scaled[train_index], X_scaled[val_index]
#     y_train_fold, y_val_fold = y.iloc[train_index], y.iloc[val_index]
    
#     # Create LightGBM model with early stopping in each fold
#     model_fold = lgb.LGBMRegressor(**best_params, n_estimators=1000)
#     model_fold.fit(X_train_fold, y_train_fold, 
#                    eval_set=[(X_val_fold, y_val_fold)], 
#                    early_stopping_rounds=100, 
#                    verbose=0)  # Suppress verbose logging during cross-validation
    
#     # Predict and evaluate
#     y_pred_fold = model_fold.predict(X_val_fold, num_iteration=model_fold.best_iteration_)
    
#     mae = mean_absolute_error(y_val_fold, y_pred_fold)
#     rmse = np.sqrt(mean_squared_error(y_val_fold, y_pred_fold))
    
#     mae_scores.append(mae)
#     rmse_scores.append(rmse)

# # Print MAE and RMSE scores
# print(f"Mean MAE: {np.mean(mae_scores):.4f}")
# print(f"Mean RMSE: {np.mean(rmse_scores):.4f}")

# # Final model prediction on the test set
# y_test_pred = best_model.predict(X_test_scaled, num_iteration=best_model.best_iteration_)

# # Save predictions in a CSV file
# output_df = pd.DataFrame({
#     'id': test['id'],  # Use the 'id' column from test data
#     'yield': y_test_pred  # Predicted 'yield'
# })
# output_df.to_csv('yield_predictions.csv', index=False)

# print("Predictions saved to 'yield_predictions.csv'")
